In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

# Define the function to create LSTM dataset and save it
def create_and_save_lstm_dataset(file_id, n_intervals, datasets_folder='../datasets', scalers_folder='../scalers'):
    file_path = f'../data_updated/{file_names[file_id]}'
    dataset = pd.read_csv(file_path).dropna()

    target_column = dataset.pop('Цена')
    dates = dataset.pop('Дата')
    dataset['Цена'] = target_column

    normalized_dataset = MinMaxScaler(feature_range=(0, 1)).fit_transform(dataset.values)

    X_ltsm, y_ltsm = create_lstm_dataset(normalized_dataset, n_intervals, forecast_days=1)

    train_size = int(len(X_ltsm) * 0.8)
    X_train_ltsm, X_test_ltsm = X_ltsm[:train_size], X_ltsm[train_size:]
    y_train_ltsm, y_test_ltsm = y_ltsm[:train_size], y_ltsm[train_size:]

    # Save scalers
    scaler = MinMaxScaler(feature_range=(0, 1)).fit(X_train_ltsm.reshape(-1, X_train_ltsm.shape[-1]))
    scaler_filename = os.path.join(scalers_folder, f'{file_names[file_id]}_scaler.pkl')
    with open(scaler_filename, 'wb') as scaler_file:
        joblib.dump(scaler, scaler_file)

    # Save datasets
    X_train_ltsm_file_path = os.path.join(datasets_folder, f'X_train_ltsm_{file_names[file_id]}.npy')
    X_test_ltsm_file_path = os.path.join(datasets_folder, f'X_test_ltsm_{file_names[file_id]}.npy')
    y_train_ltsm_file_path = os.path.join(datasets_folder, f'y_train_ltsm_{file_names[file_id]}.npy')
    y_test_ltsm_file_path = os.path.join(datasets_folder, f'y_test_ltsm_{file_names[file_id]}.npy')

    np.save(X_train_ltsm_file_path, X_train_ltsm)
    np.save(X_test_ltsm_file_path, X_test_ltsm)
    np.save(y_train_ltsm_file_path, y_train_ltsm)
    np.save(y_test_ltsm_file_path, y_test_ltsm)

    print(f'Data saved to {X_train_ltsm_file_path} and {X_test_ltsm_file_path}')
    print(f'Data saved to {y_train_ltsm_file_path} and {y_test_ltsm_file_path}')

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import sqrt
from keras.losses import mean_squared_error

from free_utils import file_names, n_intervals

# Функция для формирования строки с описанием архитектуры модели
def get_model_architecture(model):
    architecture = ""
    for layer in model.layers:
        if isinstance(layer, LSTM):
            architecture += f"LSTM_{layer.units}x{layer.return_sequences}_"
        elif isinstance(layer, Dense):
            architecture += f"Dense{layer.units}_"
    return architecture[:-1]

# Функция для обучения LSTM модели и прогнозирования на следующие 300 дней
def train_lstm_model_and_forecast(file_id, n_intervals, n_features, models_dir='models', data_folder='../datasets'):
    # Загрузка данных
    scaler_filename = '../scalers/' + file_names[file_id] + '_scaler.pkl'
    with open(scaler_filename, 'rb') as scaler_file:
        scaler = joblib.load(scaler_file)

    file_paths = ['X_test_ltsm_', 'X_train_ltsm_', 'y_test_ltsm_', 'y_train_ltsm_']
    data = []

    for file_path in file_paths:
        file_name = file_path + file_names[file_id] + '.npy'
        full_path = os.path.join(data_folder, file_name)
        data.append(np.load(full_path))

    X_test_ltsm, X_train_ltsm, y_test_ltsm, y_train_ltsm = data

    # Создание модели
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train_ltsm.shape[1], X_train_ltsm.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    model.compile(optimizer='adam', loss='mse')

    # Определение колбеков
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_filename = os.path.join(models_dir, f'{file_names[file_id]}_{get_model_architecture(model)}_best_model.h5')
    model_checkpoint = ModelCheckpoint(model_filename, monitor='val_loss', save_best_only=True)

    # Обучение модели
    history = model.fit(X_train_ltsm, y_train_ltsm,
                        epochs=30, batch_size=32,
                        validation_data=(X_test_ltsm, y_test_ltsm),
                        callbacks=[early_stopping, model_checkpoint])

    # Визуализация процесса обучения
    plt.plot(history.history['loss'][3:], label='Train Loss')
    plt.plot(history.history['val_loss'][3:], label='Validation Loss')
    plt.legend()
    plt.show()

    # Прогноз на следующие 300 дней
    forecast_days = n_intervals
    forecast = []
    last_60_days = X_test_ltsm[-n_intervals:]

    for i in range(forecast_days):
        next_day_prediction = model.predict(last_60_days)
        forecast.append(next_day_prediction[0, 0])
        last_60_days = np.roll(last_60_days, shift=-1, axis=0)
        last_60_days[-1] = next_day_prediction

    # Обратное масштабирование прогнозов
    forecast = np.array(forecast)
    forecast = np.repeat(forecast, n_features).reshape(-1, n_features)
    forecast = scaler.inverse_transform(forecast)[:, 0]

    # Результаты прогнозирования
    print("Прогнозы на следующие 300 дней:")
    print(forecast)

    # Визуализация результатов
    plt.figure(figsize=(12, 6))
    plt.plot(range(len(inv_y_train), len(inv_y_train) + len(inv_y_test)), inv_y_test, label='Actual (Test)')
    plt.plot(range(len(inv_y_train), len(inv_y_train) + len(inv_y_test)), inv_yhat, label='Predicted (Test)')
    plt.plot(range(len(inv_y_train) + len(inv_y_test), len(inv_y_train) + len(inv_y_test) + len(forecast)), forecast, label='Forecast (Next 100 days)')
    plt.xlabel('Day')
    plt.ylabel('Price')
    plt.legend()
    plt.show()




In [ ]:
# Usage example
file_id = 4
n_intervals = 60
n_features = 18
create_and_save_lstm_dataset(file_id, n_intervals)
# Использование функции
train_lstm_model_and_forecast(file_id, n_intervals, n_features)
